In [ ]:
%load_ext lab_black
from pandas import read_excel
from sqlalchemy.future import select

from cmt_statistics_tool.insert.people import agg_name
from cmt_statistics_tool.statistics import get_data
from cmt_statistics_tool.tables import (
    People,
    RevisionPeople,
    SubmissionPeople,
    SubmissionReview,
)
from cmt_statistics_tool.tables.people import PeoplePaperRelation as ppr

In [ ]:

all_reviewers = set(r[0] for r in await get_data(statement))statement = (
    select(People.name.label("name"))
    .join(
        select(
            (
                select(SubmissionPeople.people_id)
                .where(
                    SubmissionPeople.relation_type.in_(
                        [ppr.REVIEWER, ppr.METAREVIEWER, ppr.SENIORMETAREVIEWER]
                    )
                )
                .union_all(
                    select(RevisionPeople.people_id).where(
                        RevisionPeople.relation_type.in_(
                            [ppr.REVIEWER, ppr.METAREVIEWER, ppr.SENIORMETAREVIEWER]
                        )
                    )
                )
                .subquery()
            ).c.people_id
        )
        .distinct()
        .subquery()
    )
    .union_all(
        select(SubmissionReview.external_reviewer.label("name"))
        .where(
            SubmissionReview.external_reviewer != "",
            SubmissionReview.external_reviewer != None,
        )
        .distinct()
    )
)
all_reviewers = set(r[0] for r in await get_data(statement))

In [ ]:
df = read_excel("data/Reviewer Management _ CMT.xlsx")[
    ["First Name", "Middle Initial (optional)", "Last Name"]
].fillna("")

In [ ]:
reviewer_board = set(
    df.agg(
        agg_name,
        axis=1,
    )
)
reviewer_board |= set()  # add the board names here
reviewer_board -= set()  # you may need to subtract some names

sorted(reviewer_board - all_reviewers)  # this should be the empty set

In [ ]:
additional_reviewers = (
    {" ".join(r.strip().split()) for r in all_reviewers - reviewer_board}
    - reviewer_board
    - {"", "-", "N.A.", "Not applicable"}
)
print("\n".join(sorted(additional_reviewers)))

In [ ]:
manually_cleaned = """"""  # insert the manually cleaned external reviewers here
manually_cleaned = set(r.strip() for r in manually_cleaned.strip().split("\n"))
final = manually_cleaned - reviewer_board
print("\n".join(sorted(final)))